In [1]:
import numpy as np
import pandas as pd
import random

random.seed(42)
np.random.seed(42)

Define EV Specifications
ev_specs = [
    {"name": "tesla_model_3", "capacity_kwh": 62, "consumption_per_100km": 14.4}, # 11.5
    {"name": "vw_id3_pro", "capacity_kwh": 58, "consumption_per_100km": 15.6},  # 11
    {"name": "vw_id7", "capacity_kwh": 77, "consumption_per_100km": 14.1}, # 11
    {"name": "bmw_i4_m50", "capacity_kwh": 83.9, "consumption_per_100km": 22.5}, # 11
    {"name": "benz_eqe_300", "capacity_kwh": 89, "consumption_per_100km": 16.4} # 11
]

# for users classification B in the paper below(average distanc per day)
min_distance = 31.5  # km/day      # https://www.researchgate.net/publication/334124582_Environmental_Sustainability_in_the_Context_of_Mass_Personalisation_-_Quantification_of_the_Carbon_Footprint_with_Life_Cycle_Assessment
max_distance = 49.3  # km/day

#Generate 4 daily_energy_kwh values per EV model
results = []

for ev in ev_specs:
    for i in range(4):  # 4 random values per model
        daily_km = np.random.uniform(min_distance, max_distance)
        daily_energy_kwh = (ev["consumption_per_100km"] / 100) * daily_km
        results.append({
            "ev_model": ev["name"],
            "daily_km": daily_km,
            "daily_energy_kwh": daily_energy_kwh
        })


df = pd.DataFrame(results)

df["consumption_per_100km [kWh]"] = np.nan  
df.loc[df["ev_model"] == "tesla_model_3", "consumption_per_100km [kWh]"] = 14.4
df.loc[df["ev_model"] == "vw_id3_pro", "consumption_per_100km [kWh]"] = 15.6
df.loc[df["ev_model"] == "vw_id7", "consumption_per_100km [kWh]"] = 14.1
df.loc[df["ev_model"] == "bmw_i4_m50", "consumption_per_100km [kWh]"] = 22.5
df.loc[df["ev_model"] == "benz_eqe_300", "consumption_per_100km [kWh]"] = 16.4

df["monthly_energy_kwh"] = df["daily_energy_kwh"] * 31
df["total hours of charging in month"] = df["monthly_energy_kwh"] / 11


In [2]:
df["charging_hours_rounded"] = np.ceil(df["total hours of charging in month"]).astype(int)
df

,ev_model,daily_km,daily_energy_kwh,consumption_per_100km [kWh],monthly_energy_kwh,total hours of charging in month,charging_hours_rounded
0,tesla_model_3,38.166814,5.496021,14.4,170.376658,15.488787,16
1,tesla_model_3,48.422715,6.972871,14.4,216.158998,19.650818,20
2,tesla_model_3,44.529492,6.412247,14.4,198.779653,18.070878,19
3,tesla_model_3,42.156121,6.070481,14.4,188.184924,17.107720,18
4,vw_id3_pro,34.277132,5.347233,15.6,165.764209,15.069474,16
5,vw_id3_pro,34.276702,5.347166,15.6,165.762133,15.069285,16
6,vw_id3_pro,32.533888,5.075287,15.6,157.333884,14.303080,15
7,vw_id3_pro,46.917935,7.319198,15.6,226.895136,20.626831,21
8,vw_id7,42.199847,5.950178,14.1,184.455532,16.768685,17
9,vw_id7,44.103692,6.218621,14.1,192.777237,17.525203,18


In [3]:
import pandas as pd
import numpy as np
snapshots = pd.date_range("2023-03-01 00:00", "2023-03-31 23:00", freq="h")


ev_data = df
#fixed 7-day intervals in March
interval_starts = pd.date_range("2023-03-01", "2023-03-31", freq="7D")
intervals = [(start, min(start + pd.Timedelta(days=6), snapshots[-1])) for start in interval_starts]

charging_profiles = pd.DataFrame(index=snapshots)

for idx, row in ev_data.iterrows():
    total_hours = int(row['charging_hours_rounded'])
    n_blocks = len(intervals)

    # Split charging hours across intervals 
    base_hours = total_hours // n_blocks
    remainder = total_hours % n_blocks
    block_hours_per_interval = [base_hours + (1 if i < remainder else 0) for i in range(n_blocks)]


    charging_series = pd.Series(0.0, index=snapshots)

    for i, (start, end) in enumerate(intervals):
        block_len = block_hours_per_interval[i]
        if block_len == 0:
            continue

        
        hours_in_interval = pd.date_range(start, end, freq='h')
        valid_hours = hours_in_interval.intersection(snapshots)

        # Allowed start hours for charging (09–12 or 19–23)
        allowed_start_hours = valid_hours[valid_hours.hour.isin([9, 10, 11, 12, 19, 20, 21, 22, 23])]

        # Filter to those with enough remaining hours for full block
        possible_starts = []
        for ts in allowed_start_hours:
            block = pd.date_range(ts, periods=block_len, freq='h')
            if all(h in allowed_start_hours for h in block):
                possible_starts.append(ts)

        if not possible_starts:
            continue 

        # Randomly select one valid starting point
        start_ts = np.random.choice(possible_starts)
        block_hours = pd.date_range(start_ts, periods=block_len, freq='h').intersection(snapshots)

        charging_series[block_hours] = 11.0

    
    charging_profiles[f'ev_{idx}'] = charging_series


In [4]:
charging_hours_per_ev = {}

for ev in charging_profiles.columns:
    # select timestamps where charging power is > 0 (i.e, charging ocurs)
    charging_hours = charging_profiles.index[charging_profiles[ev] > 0]
    charging_hours_per_ev[ev] = charging_hours

# Fixed charging profile

In [5]:
import pandas as pd
import numpy as np

ev_data = pd.DataFrame({
    'ev_model': ['tesla_model_3', 'tesla_model_3', 'tesla_model_3', 'tesla_model_3',
                 'vw_id3_pro', 'vw_id3_pro', 'vw_id3_pro', 'vw_id3_pro',
                 'vw_id7', 'vw_id7', 'vw_id7', 'vw_id7',
                 'bmw_i4_m50', 'bmw_i4_m50', 'bmw_i4_m50', 'bmw_i4_m50',
                 'benz_eqe_300', 'benz_eqe_300', 'benz_eqe_300', 'benz_eqe_300'],
    'charging_hours_rounded': [16, 20, 19, 18, 16, 16, 15, 21,
                                17, 18, 13, 20, 30, 23, 23, 23,
                                18, 19, 19, 17]
})

snapshots = pd.date_range("2023-03-01 00:00", "2023-03-31 23:00", freq="h")

# 7-day intervals for charging distribution
interval_starts = pd.date_range("2023-03-01", "2023-03-31", freq="7D")
intervals = [(start, min(start + pd.Timedelta(days=6), snapshots[-1])) for start in interval_starts]

charging_profiles = pd.DataFrame(index=snapshots)

model_counter = {}

# Generate charging profiles
for idx, row in ev_data.iterrows():
    total_hours = int(row['charging_hours_rounded'])
    ev_model = row['ev_model']
    n_blocks = len(intervals)

    # Distribute hours across to the blocks
    base_hours = total_hours // n_blocks
    remainder = total_hours % n_blocks
    block_hours_per_interval = [base_hours + (1 if i < remainder else 0) for i in range(n_blocks)]

    
    charging_series = pd.Series(0.0, index=snapshots)

    for i, (start, end) in enumerate(intervals):
        block_len = block_hours_per_interval[i]
        if block_len == 0:
            continue

        # Allowed hours
        hours_in_interval = pd.date_range(start, end, freq='h')
        valid_hours = hours_in_interval.intersection(snapshots)
        allowed_start_hours = valid_hours[valid_hours.hour.isin([9, 10, 11, 12, 19, 20, 21, 22, 23])]

        # find valid starting points for consecutive blocks
        possible_starts = []
        for ts in allowed_start_hours:
            block = pd.date_range(ts, periods=block_len, freq='h')
            if all(h in allowed_start_hours for h in block):
                possible_starts.append(ts)

        if not possible_starts:
            continue

        start_ts = np.random.choice(possible_starts)
        block_hours = pd.date_range(start_ts, periods=block_len, freq='h').intersection(snapshots)

        charging_series[block_hours] = 11.0

    
    model_counter[ev_model] = model_counter.get(ev_model, 0) + 1
    ev_name = f"{ev_model} ({model_counter[ev_model]})"
    charging_profiles[ev_name] = charging_series


In [6]:
charging_profiles["tesla_model_3 (1)"].sum() / 1000

np.float64(0.176)

In [7]:
#charging_profiles.to_csv("charging_profiles.csv")


# Load driving profiles 

In [8]:
import pandas as pd
import numpy as np

charging_profiles = pd.read_csv("charging_profiles.csv", index_col=0, parse_dates=True)
charging_hours = list(range(9, 13)) + list(range(19, 24))
driving_profiles = pd.DataFrame(index=charging_profiles.index)


rng = np.random.default_rng(seed=42)  

for ev in charging_profiles.columns:
    total_energy_mwh = charging_profiles[ev].sum() / 1000  # Convert kWh to MWh
    timestamps = charging_profiles.index
    days = sorted(set(timestamps.date))
    
    driving_hours_per_ev = []

    for day in days:
        # Morning : random start between 6–8 AM, 1-hour block)
        morning_hour = rng.integers(6, 9)
        # Evening : random start between 16–18 PM, 1-hour block)
        evening_hour = rng.integers(16, 19)
        
        
        # Create full datetime entries for those hours
        date_str = pd.Timestamp(day).strftime('%Y-%m-%d')
        driving_hours_per_ev.extend([
            pd.Timestamp(f"{date_str} {morning_hour:02d}:00"),
            pd.Timestamp(f"{date_str} {evening_hour:02d}:00")
           
        ])
    
    
    valid_driving_hours = [t for t in driving_hours_per_ev if t in timestamps]
    
    
    energy_per_hour = total_energy_mwh / len(valid_driving_hours)
    profile = pd.Series(0.0, index=timestamps)
    profile.loc[valid_driving_hours] = energy_per_hour
    
    driving_profiles[ev] = profile




In [9]:
#driving_profiles.to_csv("driving_profiles.csv")

In [10]:
#driving_profiles.head(50)

# driving profile 2

In [11]:
import pandas as pd
import numpy as np

charging_profiles = pd.read_csv("charging_profiles.csv", index_col=0, parse_dates=True)

driving_profiles_2 = pd.DataFrame(index=charging_profiles.index)

rng = np.random.default_rng(seed=42)

timestamps = charging_profiles.index
days = sorted(set(timestamps.date))

for ev in charging_profiles.columns:
    total_monthly_energy_mwh = charging_profiles[ev].sum() / 1000  # kWh to MWh
    avg_daily_energy = total_monthly_energy_mwh / len(days)
    
    profile = pd.Series(0.0, index=timestamps)

    for day in days:
        #daily energy with ±20% variation
        daily_energy = rng.uniform(0.8, 1.2) * avg_daily_energy
        
        #random split: 35–45% morning, 55–65% evening
        morning_share = rng.uniform(0.35, 0.45)
        evening_share = 1 - morning_share

        #random driving hours
        morning_hour = rng.integers(6, 10)
        evening_hour = rng.integers(16, 19)

        #create datetime strings
        date_str = pd.Timestamp(day).strftime('%Y-%m-%d')
        morning_time = pd.Timestamp(f"{date_str} {morning_hour:02d}:00")
        evening_time = pd.Timestamp(f"{date_str} {evening_hour:02d}:00")

        #assign to profile (only if timestamps are valid)
        if morning_time in profile.index:
            profile.loc[morning_time] = morning_share * daily_energy
        if evening_time in profile.index:
            profile.loc[evening_time] = evening_share * daily_energy

    driving_profiles_2[ev] = profile


driving_profiles_2.head(50)

,tesla_model_3 (1),tesla_model_3 (2),tesla_model_3 (3),tesla_model_3 (4),vw_id3_pro (1),vw_id3_pro (2),vw_id3_pro (3),vw_id3_pro (4),vw_id7 (1),vw_id7 (2),vw_id7 (3),vw_id7 (4),bmw_i4_m50 (1),bmw_i4_m50 (2),bmw_i4_m50 (3),bmw_i4_m50 (4),benz_eqe_300 (1),benz_eqe_300 (2),benz_eqe_300 (3),benz_eqe_300 (4)
2023-03-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-03-01 01:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-03-01 02:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-03-01 03:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-03-01 04:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-03-01 05:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-03-01 06:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002199,0.000000,0.002414,0.000000,0.000000,0.002989,0.0,0.003362,0.000000,0.002840,0.000000,0.000000,0.003329,0.000000
2023-03-01 07:00:00,0.002481,0.000000,0.000000,0.002581,0.002048,0.002544,0.000000,0.003600,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-03-01 08:00:00,0.000000,0.002569,0.002301,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.003380,0.000000,0.002942,0.002435,0.000000,0.001818
2023-03-01 09:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002987,0.001640,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [12]:
#driving_profiles_2.to_csv("driving_profiles_2.csv")

# availibility 

In [13]:
import pandas as pd
import numpy as np

driving_profiles_2 = pd.read_csv("driving_profiles_2.csv", index_col=0, parse_dates=True)
availability_profiles_3 = pd.DataFrame(0, index=driving_profiles_2.index, columns=driving_profiles_2.columns)

rng = np.random.default_rng(seed=42)

# generate availability profile for each EVs
for ev in driving_profiles_2.columns:
    ev_driving = driving_profiles_2[ev]
    availability = pd.Series(0, index=ev_driving.index)

    # Group by day
    for day, group in ev_driving.groupby(ev_driving.index.date):
        driving_hours = group[group > 0].index

        if len(driving_hours) == 0:
            continue

        earliest = driving_hours.min()
        latest = driving_hours.max()

        #first window: 4 hours between first and last driving hour (but not overlapping with driving)
        full_range = pd.date_range(start=earliest, end=latest, freq='h')
        candidates_mid = []

        for h in full_range:
            candidate_window = pd.date_range(start=h, periods=5, freq='h')
            if candidate_window[-1] <= latest and all(ev_driving[candidate_window] == 0):
                candidates_mid.append(h)

        if candidates_mid:
            start_mid = rng.choice(candidates_mid)
            availability[start_mid:start_mid + pd.Timedelta(hours=4)] = 1

        #second window: 3 hours after last driving hour, starting at least 2h later
        after_range = pd.date_range(
            start=latest + pd.Timedelta(hours=2),
            end=pd.Timestamp(day) + pd.Timedelta(hours=23),
            freq='h'
        )
        candidates_after = []

        for h in after_range:
            candidate_window = pd.date_range(start=h, periods=3, freq='h')
            if candidate_window[-1] <= after_range[-1] and all(ev_driving[candidate_window] == 0):
                candidates_after.append(h)

        if candidates_after:
            start_after = rng.choice(candidates_after)
            availability[start_after:start_after + pd.Timedelta(hours=2)] = 1

    availability_profiles_3[ev] = availability




In [14]:
#availability_profiles_3.head(50)

In [15]:

#availability_profiles_3.to_csv("availability_profiles_3.csv")

# e_nom  and   e_initial 

In [16]:
import pandas as pd
charging_profiles = pd.read_csv("charging_profiles.csv", index_col=0, parse_dates=True)

# Define EV battery capacities based on model
battery_capacity_dict = {
    "tesla_model_3": 62,
    "vw_id3_pro": 58,
    "vw_id7": 77,
    "bmw_i4_m50": 83.9,
    "benz_eqe_300": 89
}

battery_data = {}
for col in charging_profiles.columns:
    
    base_model = col.split(" (")[0]
    capacity_kwh = battery_capacity_dict.get(base_model, None)
    if capacity_kwh:
        battery_data[col] = {
            "e_nom_MWh": round(capacity_kwh / 1000, 5),         # convert to MWh
            "e_initial_MWh": round(0.3 * capacity_kwh / 1000, 5)  # 20% of battery in MWh
        }

battery_df = pd.DataFrame.from_dict(battery_data, orient="index")

battery_df

,e_nom_MWh,e_initial_MWh
tesla_model_3 (1),0.0620,0.01860
tesla_model_3 (2),0.0620,0.01860
tesla_model_3 (3),0.0620,0.01860
tesla_model_3 (4),0.0620,0.01860
vw_id3_pro (1),0.0580,0.01740
vw_id3_pro (2),0.0580,0.01740
vw_id3_pro (3),0.0580,0.01740
vw_id3_pro (4),0.0580,0.01740
vw_id7 (1),0.0770,0.02310
vw_id7 (2),0.0770,0.02310


In [17]:
#battery_df.to_csv("battery_df.csv")

In [18]:
display(driving_profiles_2["tesla_model_3 (2)"].sum())
display(charging_profiles["tesla_model_3 (2)"].sum())


np.float64(0.2182928240545498)

np.float64(220.0)